# 31. 고차 미분(이론편)

- 이전 단계에서는 현재의 DeZero 구현을 되돌아 봤습니다. 요점은 다음과 같습니다. 
    - 계산의 연결은 Function 클래스 \_\_call\_\_ 메서드에서 만들어집니다. 
    - 구체적인 순전파와 역전파 계산은 Function 클래스를 상속한 클래스의 forward 메서드와 backward 메서드로 처리합니다. 
- 여기서 주목할 점은 계산 그래프의 '연결'이 만들어 지는 시점입니다. 
    - 순전파를 계산할 때 만들어집니다. 
    - 역전파를 계산할 때는 만들어지지 않는데, 여기에 문제의 핵심이 있습니다. 

## 31.1 역전파 계산

- 순전파와 마찬가지로 역전파에도 구체적인 계산 로직이 있습니다. 
- 예를 들어 이전 단계에서 본 Sin 클래스의 backward 메서드 구현은 다음과 같습니다. 

In [ ]:
class Sin(Function):
    ...
    
    def backward(self, py):
        x = self.inputs[0].data
        gx = gy * np.cos(x) # (1)
        return gx           

- 위 (1)식과 같이 구체적인 계산이 이루어지지만, 현재의 DeZero는 이 계산과 관련된 아무런 계산 그래프도 만들지 않음
- 왜냐하면 이 계산에서는 ndarray 인스턴스가 사용되기 때문임
- 만약 역전파를 계산할 때도 '연결'이 만들어진다면 어떤 장점이 있을까요?
    - 바로 고차 미분을 자동으로 계산할 수 있게 됩니다!
- 아래 계산 그래프를 살펴 봅시다.   

![title](image/그림31-1.png)

- 위는 y = sin(x)의 계산 그래프입니다. 이 때 y.backward()를 호출하면 y의 x에 대한 미분이 구해집니다. 
- 아래 계산 그래프를 살펴봅시다. 

![title](image/그림31-2.png)

- 위는 'sin 함수의 미분'을 구하기 위한 계산 그래프입니다. 
- 이 계산 그래프는 sin 클래스의 역전파 코드 (gx = gy * np.cos(x))를 계산 그래프로 나타낸 것입니다. 
- 만약 위 그림과 같은 계산 그래프가 있다면 gx.backward()를 호출하여 gx의 x에 대한 미분을 계산할 수 있습니다. 
- 원래 gx는 y = sin(x)의 미분이기 때문에 gx.backward()를 호출함으로써 x에 대한 미분이 한 번 더 이루어집니다. 
- 즉, 이것이 x의 2차 미분에 해당합니다. 
    - x를 시간으로 gx를 가속도로 바꿔서 생각해 보면, 위 그림은 '시간'을 입력하면 '속도'가 출력되는 계산그래프가 됨
    - 이 때 역전파를 하면 '속도'의 '시간'에 대한 미분(변화)가 구해지는데, 바로 '가속도'에 해당함
- 앞으로의 목표는 위 그림과 같은 '미분 계산'을 계산 그래프로 만드는 것입니다. 
- 여기서 '미분 계산'이라 함은, 다시 말해 역전파 때 수행되는 계산입니다. 
- 따라서 역전파 때 수행되는 계산에 대해서도 '연결'을 만들면 문제가 해결됩니다. 
- 이제 이 아이디어를 현실화 하는 방법을 생각해 보겠습니다. 

## 31.2 역전파로 계산 그래프 만들기

- DeZero는 순전파로 계산의 '연결'을 만듭니다. 
- 더 정확하게는 'Variable 인스턴스를 사용'하여 일반적인 계산(순전파)을 하는 시점에 '연결'이 만들어 집니다. 
- 이 말은 곧, 함수의 backward 메서드에서도 ndarray 인스턴스가 아닌 Variable 인스턴스를 사용하면 계산의 '연결'이 만들어진다는 뜻입니다. 
- 이를 위한 준비로 우선 미분값(기울기)을 Variable 인스턴스 형태로 유지해야 합니다. 
- 즉, Variable을 아래 그림처럼 변경합니다. 

![title](image/그림31-3.png)

- 위 그림에서 알 수 있듯이 지금까지 Variable 클래스의 grad는 ndarray 인스턴스를 참조했습니다. 
- 이를 Variable 인스턴스를 참조하도록 변경합니다. 
- 이렇게 변경하면 y = sin(x)를 아래와 같은 계산 그래프로 표현할 수 있습니다. 

![title](image/그림31-4.png)

- 위 그림은 Sin 클래스의 순전파와 역전파를 수행한 후의 계산 그래프입니다. 
- 중요한 점은 역전파 계산이 대한 계산 그래프도 만들어진다는 것입니다. 
- 미분값을 나타내는 gy가 Variable 인스턴스가 된 덕분에 gy를 사용한 계산에도 '연결'이 만들어지는 것입니다. 
    - y = sin(x) 계산에서 y.backward()를 수행하면 x와 같은 '말단 변수(사용자가 제공한 변수)'만 미분값을 유지합니다. 
    - y는 함수가 만들어내는 변수이므로 미분값을 유지하지 않기 때문입니다. 
    - 위 그림에서도 y.grad에서 gy로의 참조는 없습니다. 
- 위 그림에서는 Sin 클래스의 역잔파 계산 내용을 생략했습니다. 
- Sin 클래스의 backward 메서드를 구현했을 때 그 미분을 계산하는 코드는 gx = gy * cos(x)였습니다. 
- 이 때늬 변수가 모두 Variable 인스턴스로 바뀌었다고 생각해보죠.
- 그러면 위에서 생략된 역전파 계산은 실제로는 아래의 계산 그래프를 만듭니다. 

![title](image/그림31-5.png)

- 위 그림은 y.backward()를 호출함으로써 '새로' 만들어지는 계산 그래프입니다. 
- 즉, '역전파에 의해 새롭게 계산 그래프가 만들어졌다'는 뜻입니다. 
- 위와 같은 계산 그래프가 있다면 gx.backward()를 호출함으로써 y의 x에 대한 2차 미분이 이루어집니다. 
- 이상이 고차 미분을 계산하기 위한 전략입니다. 
- 다음 단계에서는 전략을 실제로 구현하겠습니다. 

# 32. 고차 미분(구현 편)

- 이번 단계에서는 고차 미분을 해낼 수 있도록 DeZero를 변경하겠습니다. 
- 이전 단계에서 이야기한 대로 역전파 시 수행되는 계산에 대해서도 계산 그래프를 만들면 됩니다. 
- 역전파 때도 Variable 인스턴스를 사용하면 해결된다는 것을 이전 단계에서 설명했으니, 이제 코드로 구현할 차례입니다. 
    - Variable 클래스를 core_simple.py에서 core.py으로 대체해서 새로 구현하겠습니다. 

## 32.1 새로운 DeZero로!

- 새로운 DeZero로 가는 가장 중요한 변화는 Variable 클래스의 인스턴스 변수인 grad 입니다. 
- 지금까지의 grad는 ndarry 인스턴스를 참조했습니다 
- 반면 새로운 DeZero에서는 다른 Variable 인스턴스를 참조하겠습니다. 
- 따라서 Variable 클래스의 아래 부분을 변경합니다. 

In [2]:
class Variable:
...

    def backward(self, retain_grad=False):
        if self.grad is None:
            # self.grad = np.ones_like(self.data)
            self.grad = Variable(np.ones_like(self.data))  # 이 부분 변경
        ...    

- 이와 같이 단 한줄만 바꾸면 됩니다. 
- 이 작은 변경 덕에 미분값을 자동으로 저장하는 코드에서 self.grad가 Variable 인스턴스를 담게 됩니다. 
- 이상으로 새로운 Variable 클래스를 완성했습니다. 

## 32.2 함수 클래스의 역전파

- 이제 DeZero의 구체적인 함수들의 backward 메서드를 수정하는 일이 남았습니다. (Function 클래스는 수정할 게 없습니다.)
- 지금까지는 core_simple.py 파일에서 다음 DeZero 함수 클래스를 구현했습니다. 
    - Add, Mul, Neg, Sub, Div, Pow
- 이번 단계에서는 이 클래스들의 backward 메서드를 수정한 다음 core.py로 옮기겠습니다. 
- 첫 번째인 Add 클래스는 수정할 게 아무것도 없습니다. Add 클래스의 구현은 아래와 같습니다. 

In [ ]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


![title](image/그림32-1a.png)

- Add 클래스의 역전파가 하는 일은 출력 쪽에서 전해지는 미분값을 입력 쪽으로 전달하는 게 다입니다. 
- 이처럼 역전파 때는 아무것도 계산하지 않기 때문에 수정할 거리도 없는 것입니다. (Sub 클래스도 마찬가지 입니다.)

![title](image/그림32-1b.png)

![title](image/그림32-1c.png)

- 위 그림과 같이 수정 전에는 Variable 인스턴스 안에 있는 데이터(ndarray 인스턴스)를 꺼내야 했습니다. 
- 한번 수정 후에는 Mul 클래스에서 Variable 인스턴스를 그대로 사용합니다. 
- 위 그림에서 주목할 점은 역전파를 계산하는 gy * x1 코드입니다. 
- 다시 말하지만 새로운 DeZero에서는 gy와 x1이 Variable 인스턴스입니다. 
- Variable 클래스의 $*$ 연산자는 이미 오버로드 되어 있으므로 gy * x1이 실행되는 뒤편에는 Mul 클래스의 순전파가 호출됩니다. 
- 그 때 Function.\_\_call\_\_()이 호출되고, 그 안에서 계산 그래프가 만들어집니다. 
    - 역전파 계산은 Variable 인스턴스에 대해 이루어집니다. 
    - 따라서 DeZero 함수에 Variable 인스턴스를 넣어 계산해야 합니다. 
- 나머지 Div, Pow 클래스의 backward 메서드는 아래를 참조하세요.     

![title](image/그림32-1d.png)

![title](image/그림32-1e.png)

## 32.3 역전파를 더 효율적으로 (모드 추가)

- 앞에서 역전파의 활성/비활성 모드를 도입했습니다. 
- 역전파가 필요 없는 경우에는 '역전파 비활성 모드'로 전환하여 역전파 처리(계산 그래프 생성과 입력 변수 유지 등)를 생략한 것입니다. 
- 이번 절에서는 역전파에서 수행하는 계산에도 같은 전략을 도입합니다. 
- 역전파를 처음 한 번 이후로 다시 할 일이 없다면 (역전파를 1회만 한다면) 역전파 계산도 '역전파 비활성 모드'로 실행하도록 하는 것입니다. 
- 이를 위해 Variable 클래스의 backward 메서드에 다음 코드를 추가합니다. 

In [16]:
def backward(self, retain_grad=False, create_graph=False): # create_graph를 추가하고 기본값 False, '역전파 비활성 모드'
    ...
    while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs]  # output is weakref

        with using_config('enable_backprop', create_graph):  # 실제 역전파의 처리 수행
            gxs = f.backward(*gys)          # 메인 backward
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx    # 이 계산도 대상

                if x.creator is not None:
                    add_func(x.creator)

- 가령 Mul 클래스의 backward 메서드는 gy * x1 계산을 합니다. 
- 여기서 '$*$ 연산자'는 오버로드되어 있기 때문에 실제로는 Mul()(gy.x1) 코드가 호출되고,
- 이어서 부모 클래스인 Function의 ```__call__()```이 호출됩니다. 
- ```Function.__call__()```메서드에서 Config.enable_backprop이 참조되고, 역전파 활성/비활성 모드가 전환됩니다.
- 또한 ```create_graph = False```로 기본 설정한 이유는 실무에서 역전파가 단 1회만 수행되는 경우가 압도적으로 많기 때문입니다. 
- 만약 2차 이상의 미분이 필요하다면 ```create_graph = True```로 설정합니다. 
- 그러면 역전파를 계산할 때도 계산 그래프가 만들어지므로 역전파를 반복해서 할 수 있습니다. 

## 32.4 \_\_init\_\_.py 변경

- 이것으로 새로운 DeZero의 핵심은 완성되었습니다. 
- 지금까지 수정을 core.py에 넣고, 앞으로 simple_core.py 대신 core.py를 사용할 것이므로
- 초기화를 수행하는 \_\_init\_\_.py를 아래처럼 수정합니다. 

In [ ]:
# =============================================================================
# step23.py부터 step32.py까지는 simple_core를 이용해야 합니다.
is_simple_core = False  # True
# =============================================================================

if is_simple_core:
    from dezero.core_simple import Variable
    from dezero.core_simple import Function
    from dezero.core_simple import using_config
    from dezero.core_simple import no_grad
    from dezero.core_simple import as_array
    from dezero.core_simple import as_variable
    from dezero.core_simple import setup_variable

else:
    from dezero.core import Variable
    from dezero.core import Parameter
    from dezero.core import Function
    from dezero.core import using_config
    from dezero.core import no_grad
    from dezero.core import test_mode
    from dezero.core import as_array
    from dezero.core import as_variable
    from dezero.core import setup_variable

- 이와 같이 ```is_simple_core = False```로 설정하여 주요 기능을 core.py에서 임포트하도록 바꿉니다. 
- 이것으로 고차 미분에 대응하는 core 파일을 임포트 할 수 있습니다. 
- 다음 단계에는 새로운 DeZero를 사용하여 고차 미분을 자동으로 계산해보겠습니다. 

# 33. 뉴턴 방법으로 푸는 최적화(자동 계산)

- 이번 단계에서는 새로운 DeZero를 사용하여 2차 미분도 자동으로 계산할 생각입닏.ㅏ 
- 우선 간단한 수식의 2차 미분을 계산하고, 미분값이 제대로 구해졌음을 확인한 다음, 이어서 뉴턴 방법을 사용해 최적화하겠습니다. 

## 33.1 2차 미분 계산하기 

- $y = x^4 - 2x^2$ 이라는 수식의 2차 미분을 계산해봅시다. 
- DeZero를 사용하면 아래와 같이 구현할 수 있습니다. (사실 한 가지 문제가 있습니다.)

In [71]:
import numpy as np
# from dezero import Variable
from dezero.core import Variable

def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)  # (1)
print(x.grad)

# 두 번째 역전파 진행
gx = x.grad   # (2)
gx.backward() # (3)
print(x.grad)

variable(24.0)
variable(68.0)


- 위 (1)의 ```y.backward(create_graph=True)```에 의해 첫번째 역전파가 진행됩니다. 이 때 인수 create_graph를 True로 지정하여 역전파 계산에 대해서도 계산 그래프를 만들게 했습니다. 
- 이제 역전파 계산 그래프에 다시 한 번 역전파할 차례입니다. 
- x의 2차 이분을 계산해야 하므로 먼저 (2)의 gx = x.grad 코드로 y의 x에 대한 미분값을 꺼냅니다. 
- 그리고 (3)에서는 방금 꺼낸 미분값인 gx에서 한번 더 역전파합니다. 
- 이렇게 하면 gx의 x에 대한 미분이 구해집니다. 
- 이 두 번째 미분이 바로 2차 미분에 해당합니다. 

- 코드를 실행하면 1차 미분이 24.0이고, 2차 alqnsdl 68.0이 나옵니다. 
- 수식으로 확인하면 $y' = 4x^3 - 4x$이므로 x = 2 일 때의 1차 미분은 24입니다. 구현 결과와 일치합니다. 
- 그런데 2차 미분은 수식으로 $y'' = 12x^2 - 4$이므로 x = 2 일 때는 44입니다. 결과가 다릅니다!

- 앞의 코드가 내 놓은 68이라는 값은 1차 미분 결과(24)에 2차 미분 결과(44)가 '더해진' 값입니다. 
- 즉, Variable에 미분값이 남아 있는 상태에서 새로운 역전파를 수행했기 때문에 새로운 미분값이 '더해진' 것입니다. 

- 이 문제를 해결하려면 새로운 계산을 하기 전에 Variable의 미분값을 '재설정'해야 합니다. 이를 반영하여 이전 문제를 다시 풀어봅시다.

- 복습! 
    - 역전파 시 ```x.backward(retain_grad = False)```처럼 retain_grad라는 인수가 False(기본값)이면 
    - 계산 과정에서 중간 변수의 미분값(기울기)은 자동으로 재설정됩니다. 
    - 따라서 말단 변수(사용자가 제공한 변수)만 미분값을 갖게 됩니다. 
    - 예컨데 x.backward()를 호출하면 x만 미분값을 유지합니다. 

In [72]:
x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)  
print(x.grad)

# 두 번째 역전파 진행
gx = x.grad   
x.cleargrad()  # 미분값 재설정
gx.backward() 
print(x.grad)

variable(24.0)
variable(44.0)


- 보시다시피 gx.backward()를 부르기 전에 x.cleargrad()를 추가했습니다. 
- 이 한 줄이 x의 미분값을 재설정하여 역전파가 올바르게 이루어지게 합니다. 
- 실제로 앞이 코드를 실행하면 2차 미분의 결과는 44.0이 나오며, 수식으로 확인한 결과와 일치합니다. 

## 33.2 뉴턴 방법을 활용한 최적화

- 이제 뉴턴 방법을 활용해 최적화를 해볼 차례입니다. 우선 뉴턴 방법을 활용한 최적화의 수식을 다시 확인해 보죠.

![title](image/식33.1.png)

- 위 식과 같이 함수 f(x)의 1차 미분과 2차 미분을 사용하여 x를 갱신하고 있습니다. 
- 이 계산을 DeZero를 사용하여 자동으로 수행되도록 해봅시다. 

In [77]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
# from dezero import Variable
from dezero.core import Variable

def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

x = Variable(np.array(2.0))
iters = 10

for i in range(iters):
    print(i, x)

    y = f(x)
    x.cleargrad()
    y.backward(create_graph=True)

    gx = x.grad
    x.cleargrad()
    gx.backward()
    gx2 = x.grad

    x.data -= gx.data / gx2.data


0 variable(2.0)
1 variable(1.4545454545454546)
2 variable(1.1510467893775467)
3 variable(1.0253259289766978)
4 variable(1.0009084519430513)
5 variable(1.0000012353089454)
6 variable(1.000000000002289)
7 variable(1.0)
8 variable(1.0)
9 variable(1.0)


- 이 코드는 29단계에서 구현한 코드에 기초하고 있습니다. 
- 지난 번에는 2차 미분을 '손으로' 계산해 하드코딩했지만, 이번에는 backward 메서드를 2번 실행하여 자동으로 계산하여 수정했습니다. 
- 위 코드의 x값의 갱신과정을 보면 불과 7회만에 최솟값 1에 도달했습니다. 
- 즉, 뉴턴 방법을 활용한 최적화를 자동으로 진행하게끔 한 구현이 올바르게 작동함을 알 수 있습니다!

# 34. sin 함수 고차 미분

- 지금까지 고차 미분에 대응하는 함수(Add, Mul, Neg, Sub, Div, Pow 클래스)를 구현하여 core.py에 추가해뒀습니다. 
- 이번 단계에서는 새로운 DeZero 함수 몇 개를 추가로 구현하겠습니다. 
- 앞으로 나오는 DeZero 함수는 functions.py에 추가하겠습니다. 
- 이 DeZero 함수들을 다른 파일에서 사용하려면 ```from dezero.functions import sin``` 형태로 임포트해야 합니다. 

## 34.1 sin 함수 구현

- 우선 고차 미분에 대응하는 새로운 Sin 클래스를 구현하겠습니다. y = sin(x)일 때 $dy/dx=cos(x)# 입니다. 
- 따라서 Sin 클래스와 sin 함수는 다음처럼 구현할 수 있습니다. 

In [79]:
import numpy as np
from dezero.core import Function

class Sin(Function):
    def forward(self, x):
        y = xp.sin(x)
        return y

    def backward(self, gy):
        x, = self.inputs
        gx = gy * cos(x)   # (1)
        return gx


def sin(x):
    return Sin()(x)

- backward 메서드에 주목해보죠. bacward 메서드 안의 모든 변수가 Variable 인스턴스인 점이 중요합니다. (forward 메서드 안의 변수는 ndarry 인스턴스 입니다.)
- 따라서 (1)의 cos(x)는 DeZero의 cos 함수입니다. 
- 이는 Sin 클래스를 구현하려면 Cos 클래스와 cos 함수가 필요하다는 뜻입니다. 

- 또한 backward 메서드 구현 시 모든 계산은 반드시 DeZero 함수를 사용해야 합니다. 
- 만약 DeZero 함수가 없다면 새로 구현해야 합니다. 
- 앞의 gy * cos(x)에는 곱셈 연산자를 오버로드해 놓았기 때문에 DeZero의 mul 함수가 호출됩니다. 

## 34.2 cos 함수 구현

- 이어서 Cos 클래스와 cos 함수를 구현하겠습니다. 
- 수식으로는 y = cos(x) 일 때, $dy/dx=-sin(x)$ 입니다. 

In [80]:
class Cos(Function):
    def forward(self, x):
        xp = cuda.get_array_module(x)
        y = xp.cos(x)
        return y

    def backward(self, gy):
        x, = self.inputs
        gx = gy * -sin(x)
        return gx


def cos(x):
    return Cos()(x)

- 이번에도 backward 메서드에 주목해야 합니다. 
- backward에서의 구체적인 계산에서 sin 함수를 사용하고 있습니다. (앞에서 sin 함수를 구현했으므로 cos함수의 구현도 끝!)

## 34.3 sin 함수 고차 미분

- 다음 차례로 sin 함수의 고차 미분입니다. 
- 이번에는 2차 미분뿐 아니라 3차 미분, 4차 미분도 계산해 보겠습니다. 

In [82]:
import numpy as np
from dezero.core import Variable
import dezero.functions as F

x = Variable(np.array(1.0))
y = F.sin(x)
y.backward(create_graph=True)

for i in range(3):
    gx = x.grad
    x.cleargrad()
    gx.backward(create_graph=True)
    print(x.grad)  # n차 미분

variable(-0.8414709848078965)
variable(-0.5403023058681398)
variable(0.8414709848078965)


In [ ]:

![title](image/그림34-1.png)

In [19]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
import matplotlib.pyplot as plt
from dezero import Variable
import dezero.functions as F

x = Variable(np.linspace(-7, 7, 200))
y = F.sin(x)
y.backward(create_graph=True)

logs = [y.data]

for i in range(3):
    logs.append(x.grad.data)
    gx = x.grad
    x.cleargrad()
    gx.backward(create_graph=True)

labels = ["y=sin(x)", "y'", "y''", "y'''"]
for i, v in enumerate(logs):
    plt.plot(x.data, logs[i], label=labels[i])
plt.legend(loc='lower right')
plt.show()


ModuleNotFoundError: No module named 'dezero.functions'

# 35. 고차 미분 계산 그래프

- 

![title](image/식35.1.png)

![title](image/그림35-1.png)

In [ ]:
## 35.1 tanh 함수 미분
- 

![title](image/식35.2.png)

![title](image/식35.3.png)

In [ ]:
## 35.2 tanh 함수 구현
- 

In [ ]:
## 35.3 고차 미분 계산 그래프 시각화
- 

In [ ]:
- 

![title](image/그림35-2.png)

![title](image/그림35-3.png)

![title](image/그림35-4.png)

![title](image/그림35-5.png)


In [ ]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import Variable
from dezero.utils import plot_dot_graph
import dezero.functions as F

x = Variable(np.array(1.0))
y = F.tanh(x)
x.name = 'x'
y.name = 'y'
y.backward(create_graph=True)

iters = 1

for i in range(iters):
    gx = x.grad
    x.cleargrad()
    gx.backward(create_graph=True)

gx = x.grad
gx.name = 'gx' + str(iters + 1)
plot_dot_graph(gx, verbose=False, to_file='tanh.png')

In [ ]:
# 36. 고차 미분 이외의 용도
- 

![title](image/식36.1.png)

![title](image/식36.2.png)

In [ ]:
## 36.1 doubl backprop의 용도
- 

In [ ]:
## 36.2 딥러닝 연구에서의 사용 예
- 

![title](image/그림36-1.png)

In [ ]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import Variable

x = Variable(np.array(2.0))
y = x ** 2
y.backward(create_graph=True)
gx = x.grad
x.cleargrad()

z = gx ** 3 + y
z.backward()
print(x.grad)

## 칼럼: 뉴턴 방법과 double backprop 보충학습

![title](image/식C.1.png)

![title](image/식C.2.png)

![title](image/식C.3.png)


![title](image/식C.4.png)